# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 11 2022 9:30AM,243940,20,ALUR855661570,Alumier Labs Inc.,Released
1,Aug 11 2022 9:30AM,243940,20,ALUR901122893,Alumier Labs Inc.,Released
2,Aug 11 2022 9:30AM,243940,20,ALUR033601103,Alumier Labs Inc.,Released
3,Aug 11 2022 9:30AM,243940,20,ALUR839285492,Alumier Labs Inc.,Released
4,Aug 11 2022 9:30AM,243940,20,ALU0007509,Alumier Labs Inc.,Released
5,Aug 11 2022 9:30AM,243940,20,ALUR620217907,Alumier Labs Inc.,Released
6,Aug 11 2022 9:30AM,243940,20,ALUR483034439,Alumier Labs Inc.,Released
7,Aug 11 2022 9:30AM,243940,20,ALU0007512,Alumier Labs Inc.,Released
8,Aug 11 2022 9:30AM,243940,20,ALUR034685312,Alumier Labs Inc.,Released
9,Aug 11 2022 9:30AM,243940,20,ALUR036893520,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,243935,Executing,2
30,243935,Released,5
31,243936,Released,3
32,243938,Released,2
33,243940,Released,35


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243934,NaN,32.0
243935,2.0,5.0
243936,NaN,3.0
243938,NaN,2.0
243940,NaN,35.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243871,10.0,4.0
243880,2.0,0.0
243897,0.0,1.0
243900,0.0,1.0
243901,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243871,10,4
243880,2,0
243897,0,1
243900,0,1
243901,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243871,10,4
1,243880,2,0
2,243897,0,1
3,243900,0,1
4,243901,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243871,10,4
1,243880,2,
2,243897,,1
3,243900,,1
4,243901,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.
35,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions
37,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation
69,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation
113,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation
163,Aug 11 2022 8:47AM,243936,10,Eywa Pharma Inc.
166,Aug 11 2022 8:46AM,243935,10,Bio-PRF
173,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation
174,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation
183,Aug 11 2022 8:30AM,243918,10,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.,,35
1,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions,,2
2,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation,,32
3,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation,,44
4,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation,,50
5,Aug 11 2022 8:47AM,243936,10,Eywa Pharma Inc.,,3
6,Aug 11 2022 8:46AM,243935,10,Bio-PRF,2,5
7,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,,1
8,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation,,9
9,Aug 11 2022 8:30AM,243918,10,SugarBear,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.,35,
1,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions,2,
2,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation,32,
3,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation,44,
4,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation,50,
5,Aug 11 2022 8:47AM,243936,10,Eywa Pharma Inc.,3,
6,Aug 11 2022 8:46AM,243935,10,Bio-PRF,5,2
7,Aug 11 2022 8:30AM,243914,10,ISDIN Corporation,1,
8,Aug 11 2022 8:30AM,243917,10,ISDIN Corporation,9,
9,Aug 11 2022 8:30AM,243918,10,SugarBear,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.,35,
1,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions,2,
2,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation,32,
3,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation,44,
4,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation,50,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.,35.0,NaN
1,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions,2.0,NaN
2,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation,32.0,NaN
3,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation,44.0,NaN
4,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation,50.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 9:30AM,243940,20,Alumier Labs Inc.,35.0,0.0
1,Aug 11 2022 9:16AM,243938,16,Sartorius Bioprocess Solutions,2.0,0.0
2,Aug 11 2022 8:54AM,243934,10,ISDIN Corporation,32.0,0.0
3,Aug 11 2022 8:53AM,243933,10,ISDIN Corporation,44.0,0.0
4,Aug 11 2022 8:53AM,243932,10,ISDIN Corporation,50.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3171030,217.0,9.0
15,731736,9.0,0.0
16,731755,4.0,5.0
17,731739,15.0,0.0
19,1219539,22.0,18.0
20,487811,39.0,10.0
21,975605,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3171030,217.0,9.0
1,15,731736,9.0,0.0
2,16,731755,4.0,5.0
3,17,731739,15.0,0.0
4,19,1219539,22.0,18.0
5,20,487811,39.0,10.0
6,21,975605,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,217.0,9.0
1,15,9.0,0.0
2,16,4.0,5.0
3,17,15.0,0.0
4,19,22.0,18.0
5,20,39.0,10.0
6,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,217.0
1,15,Released,9.0
2,16,Released,4.0
3,17,Released,15.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,20,21
Status,,,,,,,
Executing,9.0,0.0,5.0,0.0,18.0,10.0,0.0
Released,217.0,9.0,4.0,15.0,22.0,39.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,20,21
0,Executing,9.0,0.0,5.0,0.0,18.0,10.0,0.0
1,Released,217.0,9.0,4.0,15.0,22.0,39.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,20,21
0,Executing,9.0,0.0,5.0,0.0,18.0,10.0,0.0
1,Released,217.0,9.0,4.0,15.0,22.0,39.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()